In [59]:
!pip install tonic --quiet
!pip install snntorch --quiet

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import tonic
import snntorch as snn
from snntorch import functional as SF

In [87]:
def train(model, criterion, optimizer, trainloader, device, num_epochs=30):
  for epoch in range(num_epochs):
      # Training loop
      model.train()
      for batch_idx, (data, target) in enumerate(trainloader):
          optimizer.zero_grad()
          spk_rec, mem_rec = model(data.to(device))
          loss = criterion(spk_rec, target.to(device))
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')

def accuracy(model, testloader, device):
    # Testing loop
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        correct2 = 0
        total2 = 0
        for data, target in testloader:
            spk_rec, mem_rec = model(data.to(device))
            print('input: ', data.size(), ' output: ', spk_rec.size())
            correct += SF.accuracy_rate(spk_rec, target.to(device)) * target.size(0)
            total += spk_rec.size(1)
            total2 += target.size(0)
            print(correct, ' out of ', target.size())

            # correct2 += (predicted == target).sum().item()

        print(f'Accuracy of the network on test set: {100 * correct / total:.2f}% ({100 * correct / total2:.2f}%)')

In [61]:
from tonic import datasets, transforms

dt = 100000

# encoding_dim = 100

transform = transforms.Compose(
            [
                # transforms.Downsample(spatial_factor=encoding_dim/700),
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                #transforms.Downsample(spatial_factor=0.057),
                transforms.ToFrame(
                    sensor_size=tonic.datasets.SHD.sensor_size,
                    # sensor_size=(40,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )

trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)

In [62]:
from tonic import DiskCachedDataset
from torch.utils.data import DataLoader

batch_size = 4096
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/'+str(dt)+'/')


trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

In [63]:
for data, labels in iter(trainloader):
  print(data.size())
  print(labels)
  break

torch.Size([10, 4096, 1, 700])
tensor([16, 19, 14,  ..., 16, 16,  6])


In [64]:
# Define the fully connected network
class FullyConnectedNetwork(nn.Module):
    def __init__(self, beta=0.95):
        super(FullyConnectedNetwork, self).__init__()
        self.fc1 = nn.Linear(700, 256)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(256, 20)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):
      spk_rec = []
      mem_rec = []

      mem1 = self.lif1.init_leaky()
      mem2 = self.lif2.init_leaky()

      for step in range(x.size(0)):
        cur1 = self.fc1(x[step].squeeze(1))
        spk1, mem1 = self.lif1(cur1, mem1)
        cur2 = self.fc2(spk1)
        spk2, mem2 = self.lif2(cur2, mem2)
        spk_rec.append(spk2)
        mem_rec.append(mem2)
      return torch.stack(spk_rec), torch.stack(mem_rec)

In [65]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 30

# Model, loss function, and optimizer
model = FullyConnectedNetwork()
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [66]:
import torch

# # Check if CUDA is available
# if torch.cuda.is_available():
#     print(f"CUDA is available. Number of devices: {torch.cuda.device_count()}")

#     for i in range(torch.cuda.device_count()):
#         print(f"Device {i}: {torch.cuda.get_device_name(i)}")
#         print(f"  Memory Allocated: {torch.cuda.memory_allocated(i)} bytes")
#         print(f"  Memory Cached: {torch.cuda.memory_reserved(i)} bytes")
#         print(f"  Memory Free: {torch.cuda.get_device_properties(i).total_memory - torch.cuda.memory_allocated(i)} bytes")
# else:
#     print("CUDA is not available.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [90]:
# Hyperparameters
learning_rate = 0.0005
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)


Epoch [1/100], Step [1/2], Loss: 3.0120
Epoch [2/100], Step [1/2], Loss: 2.9849
Epoch [3/100], Step [1/2], Loss: 2.9618
Epoch [4/100], Step [1/2], Loss: 2.9498
Epoch [5/100], Step [1/2], Loss: 2.9286
Epoch [6/100], Step [1/2], Loss: 2.9081
Epoch [7/100], Step [1/2], Loss: 2.8873
Epoch [8/100], Step [1/2], Loss: 2.8729
Epoch [9/100], Step [1/2], Loss: 2.8575
Epoch [10/100], Step [1/2], Loss: 2.8337
Epoch [11/100], Step [1/2], Loss: 2.8106
Epoch [12/100], Step [1/2], Loss: 2.7967
Epoch [13/100], Step [1/2], Loss: 2.7760
Epoch [14/100], Step [1/2], Loss: 2.7457
Epoch [15/100], Step [1/2], Loss: 2.7274
Epoch [16/100], Step [1/2], Loss: 2.7148
Epoch [17/100], Step [1/2], Loss: 2.6933
Epoch [18/100], Step [1/2], Loss: 2.6818
Epoch [19/100], Step [1/2], Loss: 2.6596
Epoch [20/100], Step [1/2], Loss: 2.6427
Epoch [21/100], Step [1/2], Loss: 2.6281
Epoch [22/100], Step [1/2], Loss: 2.6185
Epoch [23/100], Step [1/2], Loss: 2.6035
Epoch [24/100], Step [1/2], Loss: 2.5890
Epoch [25/100], Step [1/2

In [91]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

Epoch [1/100], Step [1/2], Loss: 3.0349
Epoch [2/100], Step [1/2], Loss: 2.9761
Epoch [3/100], Step [1/2], Loss: 2.9439
Epoch [4/100], Step [1/2], Loss: 2.9132
Epoch [5/100], Step [1/2], Loss: 2.8830
Epoch [6/100], Step [1/2], Loss: 2.8586
Epoch [7/100], Step [1/2], Loss: 2.8203
Epoch [8/100], Step [1/2], Loss: 2.7884
Epoch [9/100], Step [1/2], Loss: 2.7602
Epoch [10/100], Step [1/2], Loss: 2.7312
Epoch [11/100], Step [1/2], Loss: 2.6985
Epoch [12/100], Step [1/2], Loss: 2.6747
Epoch [13/100], Step [1/2], Loss: 2.6479
Epoch [14/100], Step [1/2], Loss: 2.6308
Epoch [15/100], Step [1/2], Loss: 2.6102
Epoch [16/100], Step [1/2], Loss: 2.5905
Epoch [17/100], Step [1/2], Loss: 2.5776
Epoch [18/100], Step [1/2], Loss: 2.5639
Epoch [19/100], Step [1/2], Loss: 2.5457
Epoch [20/100], Step [1/2], Loss: 2.5390
Epoch [21/100], Step [1/2], Loss: 2.5266
Epoch [22/100], Step [1/2], Loss: 2.5201
Epoch [23/100], Step [1/2], Loss: 2.5105
Epoch [24/100], Step [1/2], Loss: 2.4998
Epoch [25/100], Step [1/2

In [101]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork(beta=0.5).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

Epoch [1/100], Step [1/1], Loss: 2.9981
Epoch [2/100], Step [1/1], Loss: 2.9983
Epoch [3/100], Step [1/1], Loss: 2.9980
Epoch [4/100], Step [1/1], Loss: 2.9943
Epoch [5/100], Step [1/1], Loss: 2.9913
Epoch [6/100], Step [1/1], Loss: 2.9879
Epoch [7/100], Step [1/1], Loss: 2.9837
Epoch [8/100], Step [1/1], Loss: 2.9790
Epoch [9/100], Step [1/1], Loss: 2.9751
Epoch [10/100], Step [1/1], Loss: 2.9721
Epoch [11/100], Step [1/1], Loss: 2.9688
Epoch [12/100], Step [1/1], Loss: 2.9660
Epoch [13/100], Step [1/1], Loss: 2.9621
Epoch [14/100], Step [1/1], Loss: 2.9581
Epoch [15/100], Step [1/1], Loss: 2.9541
Epoch [16/100], Step [1/1], Loss: 2.9493
Epoch [17/100], Step [1/1], Loss: 2.9451
Epoch [18/100], Step [1/1], Loss: 2.9403
Epoch [19/100], Step [1/1], Loss: 2.9367
Epoch [20/100], Step [1/1], Loss: 2.9332
Epoch [21/100], Step [1/1], Loss: 2.9298
Epoch [22/100], Step [1/1], Loss: 2.9256
Epoch [23/100], Step [1/1], Loss: 2.9221
Epoch [24/100], Step [1/1], Loss: 2.9187
Epoch [25/100], Step [1/1

In [102]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 750

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

Epoch [1/750], Step [1/1], Loss: 3.0402
Epoch [2/750], Step [1/1], Loss: 2.9988
Epoch [3/750], Step [1/1], Loss: 2.9923
Epoch [4/750], Step [1/1], Loss: 2.9835
Epoch [5/750], Step [1/1], Loss: 2.9726
Epoch [6/750], Step [1/1], Loss: 2.9647
Epoch [7/750], Step [1/1], Loss: 2.9553
Epoch [8/750], Step [1/1], Loss: 2.9442
Epoch [9/750], Step [1/1], Loss: 2.9318
Epoch [10/750], Step [1/1], Loss: 2.9198
Epoch [11/750], Step [1/1], Loss: 2.9088
Epoch [12/750], Step [1/1], Loss: 2.8962
Epoch [13/750], Step [1/1], Loss: 2.8820
Epoch [14/750], Step [1/1], Loss: 2.8684
Epoch [15/750], Step [1/1], Loss: 2.8543
Epoch [16/750], Step [1/1], Loss: 2.8384
Epoch [17/750], Step [1/1], Loss: 2.8227
Epoch [18/750], Step [1/1], Loss: 2.8066
Epoch [19/750], Step [1/1], Loss: 2.7912
Epoch [20/750], Step [1/1], Loss: 2.7764
Epoch [21/750], Step [1/1], Loss: 2.7633
Epoch [22/750], Step [1/1], Loss: 2.7502
Epoch [23/750], Step [1/1], Loss: 2.7377
Epoch [24/750], Step [1/1], Loss: 2.7254
Epoch [25/750], Step [1/1

In [92]:
batch_size = 4096
dt = 10_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.ToFrame(
                    sensor_size=tonic.datasets.SHD.sensor_size,
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

torch.Size([100, 4096, 1, 700]) torch.Size([4096]) tensor([ 3, 17,  0,  ...,  8, 10,  3])
Epoch [1/100], Step [1/2], Loss: 3.0022
Epoch [2/100], Step [1/2], Loss: 3.0007
Epoch [3/100], Step [1/2], Loss: 2.9964
Epoch [4/100], Step [1/2], Loss: 2.9917
Epoch [5/100], Step [1/2], Loss: 2.9790
Epoch [6/100], Step [1/2], Loss: 2.9673
Epoch [7/100], Step [1/2], Loss: 2.9600
Epoch [8/100], Step [1/2], Loss: 2.9505
Epoch [9/100], Step [1/2], Loss: 2.9433
Epoch [10/100], Step [1/2], Loss: 2.9335
Epoch [11/100], Step [1/2], Loss: 2.9281
Epoch [12/100], Step [1/2], Loss: 2.9217
Epoch [13/100], Step [1/2], Loss: 2.9128
Epoch [14/100], Step [1/2], Loss: 2.9073
Epoch [15/100], Step [1/2], Loss: 2.9046
Epoch [16/100], Step [1/2], Loss: 2.8967
Epoch [17/100], Step [1/2], Loss: 2.8886
Epoch [18/100], Step [1/2], Loss: 2.8832
Epoch [19/100], Step [1/2], Loss: 2.8761
Epoch [20/100], Step [1/2], Loss: 2.8693
Epoch [21/100], Step [1/2], Loss: 2.8599
Epoch [22/100], Step [1/2], Loss: 2.8539
Epoch [23/100], S

In [98]:
batch_size = 8192
dt = 50_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.ToFrame(
                    sensor_size=tonic.datasets.SHD.sensor_size,
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

torch.Size([20, 8156, 1, 700]) torch.Size([8156]) tensor([ 9, 16,  3,  ..., 17,  5, 19])
Epoch [1/100], Step [1/1], Loss: 3.0150
Epoch [2/100], Step [1/1], Loss: 2.9980
Epoch [3/100], Step [1/1], Loss: 2.9966
Epoch [4/100], Step [1/1], Loss: 2.9948
Epoch [5/100], Step [1/1], Loss: 2.9901
Epoch [6/100], Step [1/1], Loss: 2.9817
Epoch [7/100], Step [1/1], Loss: 2.9744
Epoch [8/100], Step [1/1], Loss: 2.9668
Epoch [9/100], Step [1/1], Loss: 2.9584
Epoch [10/100], Step [1/1], Loss: 2.9504
Epoch [11/100], Step [1/1], Loss: 2.9392
Epoch [12/100], Step [1/1], Loss: 2.9243
Epoch [13/100], Step [1/1], Loss: 2.9096
Epoch [14/100], Step [1/1], Loss: 2.8958
Epoch [15/100], Step [1/1], Loss: 2.8817
Epoch [16/100], Step [1/1], Loss: 2.8671
Epoch [17/100], Step [1/1], Loss: 2.8529
Epoch [18/100], Step [1/1], Loss: 2.8391
Epoch [19/100], Step [1/1], Loss: 2.8261
Epoch [20/100], Step [1/1], Loss: 2.8130
Epoch [21/100], Step [1/1], Loss: 2.7991
Epoch [22/100], Step [1/1], Loss: 2.7866
Epoch [23/100], St

In [100]:
batch_size = 8192
dt = 50_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.ToFrame(
                    sensor_size=tonic.datasets.SHD.sensor_size,
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.003
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

torch.Size([20, 8156, 1, 700]) torch.Size([8156]) tensor([11,  3, 11,  ...,  7,  4,  2])
Epoch [1/100], Step [1/1], Loss: 3.0242
Epoch [2/100], Step [1/1], Loss: 3.0057
Epoch [3/100], Step [1/1], Loss: 2.9977
Epoch [4/100], Step [1/1], Loss: 2.9890
Epoch [5/100], Step [1/1], Loss: 2.9893
Epoch [6/100], Step [1/1], Loss: 2.9845
Epoch [7/100], Step [1/1], Loss: 2.9751
Epoch [8/100], Step [1/1], Loss: 2.9626
Epoch [9/100], Step [1/1], Loss: 2.9523
Epoch [10/100], Step [1/1], Loss: 2.9451
Epoch [11/100], Step [1/1], Loss: 2.9319
Epoch [12/100], Step [1/1], Loss: 2.9169
Epoch [13/100], Step [1/1], Loss: 2.9008
Epoch [14/100], Step [1/1], Loss: 2.8863
Epoch [15/100], Step [1/1], Loss: 2.8715
Epoch [16/100], Step [1/1], Loss: 2.8552
Epoch [17/100], Step [1/1], Loss: 2.8387
Epoch [18/100], Step [1/1], Loss: 2.8227
Epoch [19/100], Step [1/1], Loss: 2.8086
Epoch [20/100], Step [1/1], Loss: 2.7953
Epoch [21/100], Step [1/1], Loss: 2.7812
Epoch [22/100], Step [1/1], Loss: 2.7638
Epoch [23/100], St

In [99]:
batch_size = 8192
dt = 50_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.ToFrame(
                    sensor_size=tonic.datasets.SHD.sensor_size,
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork(beta=0.5).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

torch.Size([20, 8156, 1, 700]) torch.Size([8156]) tensor([14, 13,  9,  ...,  4,  9, 18])
Epoch [1/100], Step [1/1], Loss: 3.0005
Epoch [2/100], Step [1/1], Loss: 2.9964
Epoch [3/100], Step [1/1], Loss: 2.9948
Epoch [4/100], Step [1/1], Loss: 2.9930
Epoch [5/100], Step [1/1], Loss: 2.9895
Epoch [6/100], Step [1/1], Loss: 2.9865
Epoch [7/100], Step [1/1], Loss: 2.9827
Epoch [8/100], Step [1/1], Loss: 2.9793
Epoch [9/100], Step [1/1], Loss: 2.9759
Epoch [10/100], Step [1/1], Loss: 2.9727
Epoch [11/100], Step [1/1], Loss: 2.9692
Epoch [12/100], Step [1/1], Loss: 2.9656
Epoch [13/100], Step [1/1], Loss: 2.9614
Epoch [14/100], Step [1/1], Loss: 2.9572
Epoch [15/100], Step [1/1], Loss: 2.9533
Epoch [16/100], Step [1/1], Loss: 2.9501
Epoch [17/100], Step [1/1], Loss: 2.9467
Epoch [18/100], Step [1/1], Loss: 2.9436
Epoch [19/100], Step [1/1], Loss: 2.9406
Epoch [20/100], Step [1/1], Loss: 2.9372
Epoch [21/100], Step [1/1], Loss: 2.9334
Epoch [22/100], Step [1/1], Loss: 2.9297
Epoch [23/100], St

KeyboardInterrupt: 

In [93]:
batch_size = 4096
dt = 1000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.ToFrame(
                    sensor_size=tonic.datasets.SHD.sensor_size,
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

torch.Size([1000, 4096, 1, 700]) torch.Size([4096]) tensor([14, 17,  2,  ...,  2, 10,  4])
Epoch [1/100], Step [1/2], Loss: 2.9966
Epoch [2/100], Step [1/2], Loss: 2.9969


KeyboardInterrupt: 

In [94]:
batch_size = 4096
dt = 1000_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.ToFrame(
                    sensor_size=tonic.datasets.SHD.sensor_size,
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 100

# Model, loss function, and optimizer
model = FullyConnectedNetwork().to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs)
accuracy(model, testloader, device)

torch.Size([1, 4096, 1, 700]) torch.Size([4096]) tensor([11, 15, 11,  ..., 14, 16, 10])
Epoch [1/100], Step [1/2], Loss: 2.9958
Epoch [2/100], Step [1/2], Loss: 2.9445
Epoch [3/100], Step [1/2], Loss: 2.8837
Epoch [4/100], Step [1/2], Loss: 2.8394
Epoch [5/100], Step [1/2], Loss: 2.8033
Epoch [6/100], Step [1/2], Loss: 2.7177
Epoch [7/100], Step [1/2], Loss: 2.6706
Epoch [8/100], Step [1/2], Loss: 2.6339
Epoch [9/100], Step [1/2], Loss: 2.5822
Epoch [10/100], Step [1/2], Loss: 2.5172
Epoch [11/100], Step [1/2], Loss: 2.5058
Epoch [12/100], Step [1/2], Loss: 2.4916
Epoch [13/100], Step [1/2], Loss: 2.4687
Epoch [14/100], Step [1/2], Loss: 2.4377
Epoch [15/100], Step [1/2], Loss: 2.4287
Epoch [16/100], Step [1/2], Loss: 2.4187
Epoch [17/100], Step [1/2], Loss: 2.4057
Epoch [18/100], Step [1/2], Loss: 2.3921
Epoch [19/100], Step [1/2], Loss: 2.3871
Epoch [20/100], Step [1/2], Loss: 2.3818
Epoch [21/100], Step [1/2], Loss: 2.3648
Epoch [22/100], Step [1/2], Loss: 2.3603
Epoch [23/100], Ste

In [67]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        spk_rec, mem_rec = model(data.to(device))
        # print(spk_rec.size(), target.size(
        loss = criterion(spk_rec, target.to(device))
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')

Epoch [1/100], Step [1/2], Loss: 3.0226
Epoch [2/100], Step [1/2], Loss: 2.9884
Epoch [3/100], Step [1/2], Loss: 2.9621
Epoch [4/100], Step [1/2], Loss: 2.9332
Epoch [5/100], Step [1/2], Loss: 2.9117
Epoch [6/100], Step [1/2], Loss: 2.8852
Epoch [7/100], Step [1/2], Loss: 2.8579
Epoch [8/100], Step [1/2], Loss: 2.8315
Epoch [9/100], Step [1/2], Loss: 2.8037
Epoch [10/100], Step [1/2], Loss: 2.7737
Epoch [11/100], Step [1/2], Loss: 2.7403
Epoch [12/100], Step [1/2], Loss: 2.7145
Epoch [13/100], Step [1/2], Loss: 2.6916
Epoch [14/100], Step [1/2], Loss: 2.6697
Epoch [15/100], Step [1/2], Loss: 2.6356
Epoch [16/100], Step [1/2], Loss: 2.6172
Epoch [17/100], Step [1/2], Loss: 2.5972
Epoch [18/100], Step [1/2], Loss: 2.5868
Epoch [19/100], Step [1/2], Loss: 2.5661
Epoch [20/100], Step [1/2], Loss: 2.5549
Epoch [21/100], Step [1/2], Loss: 2.5450
Epoch [22/100], Step [1/2], Loss: 2.5344
Epoch [23/100], Step [1/2], Loss: 2.5255
Epoch [24/100], Step [1/2], Loss: 2.5134
Epoch [25/100], Step [1/2

In [68]:
# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, target in testloader:
        spk_rec, mem_rec = model(data.to(device))
        correct += SF.accuracy_rate(spk_rec, target.to(device)) * batch_size
        total += spk_rec.size(1)

        # total += target.size(0)
        # correct += (predicted == target).sum().item()

    print(f'Accuracy of the network on test set: {100 * correct / total:.2f}%')

Accuracy of the network on test set: 101.57%


In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 0.003
num_epochs = 30

# Model, loss function, and optimizer
model = FullyConnectedNetwork(beta=0.95).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        spk_rec, mem_rec = model(data.to(device))
        loss = criterion(spk_rec, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, target in testloader:
        spk_rec, mem_rec = model(data.to(device))
        correct += SF.accuracy_rate(spk_rec, target.to(device)) * batch_size
        total += spk_rec.size(1)

        # total += target.size(0)
        # correct += (predicted == target).sum().item()

    print(f'Accuracy of the network on test set: {100 * correct / total:.2f}%')

Epoch [1/30], Step [1/2], Loss: 3.0317
Epoch [2/30], Step [1/2], Loss: 2.9938
Epoch [3/30], Step [1/2], Loss: 2.9762
Epoch [4/30], Step [1/2], Loss: 2.9536
Epoch [5/30], Step [1/2], Loss: 2.9390
Epoch [6/30], Step [1/2], Loss: 2.9167
Epoch [7/30], Step [1/2], Loss: 2.8875
Epoch [8/30], Step [1/2], Loss: 2.8673
Epoch [9/30], Step [1/2], Loss: 2.8454
Epoch [10/30], Step [1/2], Loss: 2.8067
Epoch [11/30], Step [1/2], Loss: 2.7812
Epoch [12/30], Step [1/2], Loss: 2.7538
Epoch [13/30], Step [1/2], Loss: 2.7185
Epoch [14/30], Step [1/2], Loss: 2.6952
Epoch [15/30], Step [1/2], Loss: 2.6819
Epoch [16/30], Step [1/2], Loss: 2.6518
Epoch [17/30], Step [1/2], Loss: 2.6339
Epoch [18/30], Step [1/2], Loss: 2.6088
Epoch [19/30], Step [1/2], Loss: 2.6068
Epoch [20/30], Step [1/2], Loss: 2.5991
Epoch [21/30], Step [1/2], Loss: 2.5772
Epoch [22/30], Step [1/2], Loss: 2.5743
Epoch [23/30], Step [1/2], Loss: 2.5697
Epoch [24/30], Step [1/2], Loss: 2.5524
Epoch [25/30], Step [1/2], Loss: 2.5364
Epoch [26

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 0.0005
num_epochs = 30

# Model, loss function, and optimizer
model = FullyConnectedNetwork(beta=0.95).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        spk_rec, mem_rec = model(data.to(device))
        loss = criterion(spk_rec, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, target in testloader:
        spk_rec, mem_rec = model(data.to(device))
        correct += SF.accuracy_rate(spk_rec, target.to(device)) * batch_size
        total += spk_rec.size(1)

        # total += target.size(0)
        # correct += (predicted == target).sum().item()

    print(f'Accuracy of the network on test set: {100 * correct / total:.2f}%')

Epoch [1/30], Step [1/2], Loss: 3.0341
Epoch [2/30], Step [1/2], Loss: 2.9839
Epoch [3/30], Step [1/2], Loss: 2.9718
Epoch [4/30], Step [1/2], Loss: 2.9489
Epoch [5/30], Step [1/2], Loss: 2.9240
Epoch [6/30], Step [1/2], Loss: 2.8999
Epoch [7/30], Step [1/2], Loss: 2.8769
Epoch [8/30], Step [1/2], Loss: 2.8414
Epoch [9/30], Step [1/2], Loss: 2.8232
Epoch [10/30], Step [1/2], Loss: 2.7882
Epoch [11/30], Step [1/2], Loss: 2.7662
Epoch [12/30], Step [1/2], Loss: 2.7370
Epoch [13/30], Step [1/2], Loss: 2.7093
Epoch [14/30], Step [1/2], Loss: 2.6877
Epoch [15/30], Step [1/2], Loss: 2.6767
Epoch [16/30], Step [1/2], Loss: 2.6558
Epoch [17/30], Step [1/2], Loss: 2.6422
Epoch [18/30], Step [1/2], Loss: 2.6261
Epoch [19/30], Step [1/2], Loss: 2.6110
Epoch [20/30], Step [1/2], Loss: 2.6055
Epoch [21/30], Step [1/2], Loss: 2.5919
Epoch [22/30], Step [1/2], Loss: 2.5833
Epoch [23/30], Step [1/2], Loss: 2.5738
Epoch [24/30], Step [1/2], Loss: 2.5633
Epoch [25/30], Step [1/2], Loss: 2.5572
Epoch [26

In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 0.001
num_epochs = 30

# Model, loss function, and optimizer
model = FullyConnectedNetwork(beta=0.9).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        spk_rec, mem_rec = model(data.to(device))
        loss = criterion(spk_rec, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, target in testloader:
        spk_rec, mem_rec = model(data.to(device))
        correct += SF.accuracy_rate(spk_rec, target.to(device)) * batch_size
        total += spk_rec.size(1)

        # total += target.size(0)
        # correct += (predicted == target).sum().item()

    print(f'Accuracy of the network on test set: {100 * correct / total:.2f}%')

Epoch [1/30], Step [1/2], Loss: 3.0187
Epoch [2/30], Step [1/2], Loss: 2.9885
Epoch [3/30], Step [1/2], Loss: 2.9662
Epoch [4/30], Step [1/2], Loss: 2.9358
Epoch [5/30], Step [1/2], Loss: 2.8963
Epoch [6/30], Step [1/2], Loss: 2.8592
Epoch [7/30], Step [1/2], Loss: 2.8289
Epoch [8/30], Step [1/2], Loss: 2.8007
Epoch [9/30], Step [1/2], Loss: 2.7678
Epoch [10/30], Step [1/2], Loss: 2.7347
Epoch [11/30], Step [1/2], Loss: 2.7241
Epoch [12/30], Step [1/2], Loss: 2.7032
Epoch [13/30], Step [1/2], Loss: 2.6823
Epoch [14/30], Step [1/2], Loss: 2.6684
Epoch [15/30], Step [1/2], Loss: 2.6506
Epoch [16/30], Step [1/2], Loss: 2.6399
Epoch [17/30], Step [1/2], Loss: 2.6199
Epoch [18/30], Step [1/2], Loss: 2.6096
Epoch [19/30], Step [1/2], Loss: 2.5909
Epoch [20/30], Step [1/2], Loss: 2.5799
Epoch [21/30], Step [1/2], Loss: 2.5757
Epoch [22/30], Step [1/2], Loss: 2.5591
Epoch [23/30], Step [1/2], Loss: 2.5469
Epoch [24/30], Step [1/2], Loss: 2.5399
Epoch [25/30], Step [1/2], Loss: 2.5296
Epoch [26

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 0.001
num_epochs = 30

# Model, loss function, and optimizer
model = FullyConnectedNetwork(beta=0.5).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        spk_rec, mem_rec = model(data.to(device))
        loss = criterion(spk_rec, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, target in testloader:
        spk_rec, mem_rec = model(data.to(device))
        correct += SF.accuracy_rate(spk_rec, target.to(device)) * batch_size
        total += spk_rec.size(1)

        # total += target.size(0)
        # correct += (predicted == target).sum().item()

    print(f'Accuracy of the network on test set: {100 * correct / total:.2f}%')

Epoch [1/30], Step [1/2], Loss: 2.9961
Epoch [2/30], Step [1/2], Loss: 2.9910
Epoch [3/30], Step [1/2], Loss: 2.9774
Epoch [4/30], Step [1/2], Loss: 2.9671
Epoch [5/30], Step [1/2], Loss: 2.9537
Epoch [6/30], Step [1/2], Loss: 2.9410
Epoch [7/30], Step [1/2], Loss: 2.9290
Epoch [8/30], Step [1/2], Loss: 2.9140
Epoch [9/30], Step [1/2], Loss: 2.8987
Epoch [10/30], Step [1/2], Loss: 2.8862
Epoch [11/30], Step [1/2], Loss: 2.8747
Epoch [12/30], Step [1/2], Loss: 2.8659
Epoch [13/30], Step [1/2], Loss: 2.8511
Epoch [14/30], Step [1/2], Loss: 2.8393
Epoch [15/30], Step [1/2], Loss: 2.8288
Epoch [16/30], Step [1/2], Loss: 2.8246
Epoch [17/30], Step [1/2], Loss: 2.8146
Epoch [18/30], Step [1/2], Loss: 2.8070
Epoch [19/30], Step [1/2], Loss: 2.7984
Epoch [20/30], Step [1/2], Loss: 2.7915
Epoch [21/30], Step [1/2], Loss: 2.7840
Epoch [22/30], Step [1/2], Loss: 2.7767
Epoch [23/30], Step [1/2], Loss: 2.7715
Epoch [24/30], Step [1/2], Loss: 2.7688
Epoch [25/30], Step [1/2], Loss: 2.7625
Epoch [26

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 0.001
num_epochs = 30

# Model, loss function, and optimizer
model = FullyConnectedNetwork(beta=0.99).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        spk_rec, mem_rec = model(data.to(device))
        loss = criterion(spk_rec, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, target in testloader:
        spk_rec, mem_rec = model(data.to(device))
        correct += SF.accuracy_rate(spk_rec, target.to(device)) * batch_size
        total += spk_rec.size(1)

        # total += target.size(0)
        # correct += (predicted == target).sum().item()

    print(f'Accuracy of the network on test set: {100 * correct / total:.2f}%')

Epoch [1/30], Step [1/2], Loss: 3.0450
Epoch [2/30], Step [1/2], Loss: 2.9869
Epoch [3/30], Step [1/2], Loss: 2.9609
Epoch [4/30], Step [1/2], Loss: 2.9250
Epoch [5/30], Step [1/2], Loss: 2.9039
Epoch [6/30], Step [1/2], Loss: 2.8758
Epoch [7/30], Step [1/2], Loss: 2.8449
Epoch [8/30], Step [1/2], Loss: 2.8153
Epoch [9/30], Step [1/2], Loss: 2.7858
Epoch [10/30], Step [1/2], Loss: 2.7500
Epoch [11/30], Step [1/2], Loss: 2.7127
Epoch [12/30], Step [1/2], Loss: 2.6903
Epoch [13/30], Step [1/2], Loss: 2.6671
Epoch [14/30], Step [1/2], Loss: 2.6454
Epoch [15/30], Step [1/2], Loss: 2.6255
Epoch [16/30], Step [1/2], Loss: 2.6037
Epoch [17/30], Step [1/2], Loss: 2.5890
Epoch [18/30], Step [1/2], Loss: 2.5704
Epoch [19/30], Step [1/2], Loss: 2.5577
Epoch [20/30], Step [1/2], Loss: 2.5428
Epoch [21/30], Step [1/2], Loss: 2.5316
Epoch [22/30], Step [1/2], Loss: 2.5223
Epoch [23/30], Step [1/2], Loss: 2.5100
Epoch [24/30], Step [1/2], Loss: 2.5043
Epoch [25/30], Step [1/2], Loss: 2.5006
Epoch [26

In [ ]:
# ----------------------------------------------------------------------------------------------------

In [81]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    total_loss = 0

    for events, labels in testloader:
        events, labels = events.to(device), labels.to(device)
        output, _ = model(torch.Tensor(events).float())

        print (output.size())
        print (output)

        sum = torch.cumsum(output, dim=1)

        predicted = torch.argmax(sum[:,-1,:], 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy_test = (correct/total)*100

print(f"Test Accuracy: {accuracy_test:.3f}%")

torch.Size([10, 2264, 20])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 1.,  ..., 1., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 1., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 1.]],

        ...,

        [[0., 0., 0.,  ..., 0., 1., 1.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 

RuntimeError: The size of tensor a (10) must match the size of tensor b (2264) at non-singleton dimension 0